## A parametric sweep of different c-rate at different temperatures

In [1]:
import pybamm as pb
import numpy as np
import matplotlib.pyplot as plt
from func_project import *

c:\ProgramData\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


## graphite model

In [2]:
'''
    Pre-simulation settings
'''
# fetch parameter values
old_set = pb.ParameterValues("Chen2020")
parameters = update_parameter_set(old_set)

# mesh settings
var_pts = {"x_n": 10, "x_s": 10, "x_p": 10, "r_n": 40, "r_p": 40}
submesh_types = pb.lithium_ion.DFN().default_submesh_types
submesh_types["negative particle"] = pb.MeshGenerator(
    pb.Exponential1DSubMesh, submesh_params={"side": "right"}
)
submesh_types["positive particle"] = pb.MeshGenerator(
    pb.Exponential1DSubMesh, submesh_params={"side": "right"}
)

# output variables that are of interest
output_variables = [
    
    "Throughput capacity [A.h]",
    "Time [h]",
    "Current [A]",
    "Voltage [V]", 

    "X-averaged negative total SEI thickness [m]",
    "Loss of capacity to negative SEI [A.h]",
    "LLI [%]",
    "Discharge capacity [A.h]",
    "X-averaged negative electrode SEI interfacial current density [A.m-2]",
    
]

In [ ]:
# coupled DFN model
model = pb.lithium_ion.DFN(options={
    # submodel options
    "thermal": "lumped",
    "SEI": "VonKolzenberg2020",
    "SEI porosity change": "true",

    # misc
    "dimensionality": 0,
    "cell geometry": "arbitrary",
})

# write the readable outputs to a text file
f = open("output_vars.txt", "w")
for v in sorted(model.variable_names()):
    f.write(f"{v}\n")
f.close()

In [ ]:
# parametric sweep of different C-rates
cycles = 5      # after this is done change to a larger number to see long term degradation effects (?)
C_rate = "2C"      # ["2C", "1C", "C/5", "C/10"]

exp=pb.Experiment(
    [
        (
            f"Charge at {C_rate} until 4.2 V",
            "Rest for 1 hour",
            f"Discharge at {C_rate} until 2.5 V",
            "Rest for 1 hour",
        )
    ]* cycles
)


# make a copy of the fetched parameters
params = parameters.copy()

# converts temperature to Kelvin
temps = list(np.asarray([-10, 0, 25, 40]) + 273.15)
#temp = 298.15   #[273.15, 298.15, 313.15]

# empty solution array
graphite_sols = []

# loop through different temperatures
for temp in temps:
    # iteratively update the temperature
    params.update({"Ambient temperature [K]": temp, "Initial temperature [K]": temp,})

    # print experiment details
    params.search("Ambient temperature [K]")
    params.search("Initial temperature [K]")
    print(f"Current experiment: {exp}\n C rate:{C_rate}\n Temperature: {temp - 273.15}$^o$C")

    sim = pb.Simulation(
        model,
        experiment=exp,
        parameter_values=params,
        submesh_types=submesh_types,
        #var_pts=var_pts,
    )

    sol = sim.solve(initial_soc=0, calc_esoh=False)
    graphite_sols.append(sol)
    sol.save(f"graphite_VK_{C_rate}C_{temp}K_{cycles}cycles.pkl")


Results for 'Ambient temperature [K]': ['Ambient temperature [K]']
Ambient temperature [K] -> 298.15
Results for 'Initial temperature [K]': ['Initial temperature [K]']
Initial temperature [K] -> 298.15
Current experiment: [('Charge at 2C until 4.2 V', 'Rest for 1 hour', 'Discharge at 2C until 2.5 V', 'Rest for 1 hour'), ('Charge at 2C until 4.2 V', 'Rest for 1 hour', 'Discharge at 2C until 2.5 V', 'Rest for 1 hour'), ('Charge at 2C until 4.2 V', 'Rest for 1 hour', 'Discharge at 2C until 2.5 V', 'Rest for 1 hour'), ('Charge at 2C until 4.2 V', 'Rest for 1 hour', 'Discharge at 2C until 2.5 V', 'Rest for 1 hour'), ('Charge at 2C until 4.2 V', 'Rest for 1 hour', 'Discharge at 2C until 2.5 V', 'Rest for 1 hour')]
 C rate:2C
 Temperature: 25.0$^o$C


c:\ProgramData\anaconda3\Lib\site-packages\pybamm\simulation.py:122: UserWarning: The default solver changed to IDAKLUSolver after the v25.4.0. release. You can swap back to the previous default by using `pybamm.CasadiSolver()` instead.
  self._solver = solver or self._model.default_solver


In [ ]:
plot_results(graphite_sols, C_rates)